In [ ]:
import pandas as pd
import time
import re
import sys
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

# --- CONFIG ---
input_file = "data_usaha_kalipuro.xlsx"
output_file = "hasil_koordinat_kalipuro_strict.xlsx"
KECAMATAN_DEFAULT = "Kalipuro"

# --- KAMUS DESA KALIPURO ---
LIST_DESA_KALIPURO = [
    "Bulusan", "Bulusari", "Gombengsari", "Kalipuro", 
    "Kelir", "Ketapang", "Klatakan", "Pesucen", "Telemung"
]

# --- BATAS WILAYAH SPESIFIK KEC. KALIPURO (STRICT) ---
# Utara: -8.05 (Watu Dodol) | Selatan: -8.21 (Batas Kota)
# Barat: 114.25 (Gunung)    | Timur: 114.46 (Selat Bali)
LAT_MIN, LAT_MAX = -8.21, -8.05
LONG_MIN, LONG_MAX = 114.25, 114.46

# --- FUNGSI BERSIH-BERSIH ---
def clean_name(raw_name):
    if pd.isna(raw_name): return ""
    clean = re.sub(r'<.*?>', '', str(raw_name))
    return " ".join(clean.split())

# --- FUNGSI DETEKSI DESA ---
def deteksi_desa_dari_alamat(alamat_text):
    if pd.isna(alamat_text): return None
    alamat_lower = alamat_text.lower()
    for desa in LIST_DESA_KALIPURO:
        if desa.lower() in alamat_lower:
            return desa
    return None 

# --- FUNGSI CARI MAPS (Updated Logic) ---
def search_google_maps(driver, keyword):
    try:
        url = f"https://www.google.com/maps/search/{keyword.replace(' ', '+')}"
        driver.get(url)
        
        # 1. Tunggu sinyal '@' (Max 8 detik)
        WebDriverWait(driver, 8).until(lambda d: "@" in d.current_url)
        
        # 2. Jeda napas
        time.sleep(1.5) 
        
        match = re.search(r"@([-?\d\.]+),([-?\d\.]+)", driver.current_url)
        if match:
            lat = float(match.group(1))
            lon = float(match.group(2))
            
            # --- VALIDASI KETAT KALIPURO ---
            if (LAT_MIN <= lat <= LAT_MAX) and (LONG_MIN <= lon <= LONG_MAX):
                return lat, lon
            else:
                # Debugging: Biar tau kenapa ditolak
                print(f"[SKIP] Koordinat ({lat}, {lon}) diluar Kalipuro.", end=" ")
                return None, None
    except:
        pass
    return None, None

# --- SETUP CHROME ---
chrome_options = Options()
# chrome_options.add_argument("--headless") 
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# --- LOAD DATA ---
try:
    df = pd.read_excel(input_file)
except:
    print("File Excel ga ketemu!")
    sys.exit()

final_lats = []
final_lons = []
keterangan_sumber = []

print(f"=== START SCRAPING: KALIPURO STRICT MODE ===")
print(f"Area Batas: Lat {LAT_MIN} s/d {LAT_MAX}, Long {LONG_MIN} s/d {LONG_MAX}")

for index, row in df.iterrows():
    nama_bersih = clean_name(row["nama_usaha"])
    alamat = str(row["alamat"]) if pd.notna(row["alamat"]) else ""
    
    # Auto Detect Desa
    desa_terdeteksi = deteksi_desa_dari_alamat(alamat)
    keyword_desa = desa_terdeteksi if desa_terdeteksi else "" 
    
    print(f"[{index+1}] {nama_bersih}...", end=" ")

    # --- LEVEL 1: CARI USAHA ---
    keyword_1 = f"{nama_bersih} {alamat} {keyword_desa} {KECAMATAN_DEFAULT} Banyuwangi"
    lat, lon = search_google_maps(driver, keyword_1)
    
    if lat:
        print("-> OK (Usaha)")
        final_lats.append(f"'{lat}")
        final_lons.append(f"'{lon}")
        keterangan_sumber.append("Akurat (Titik Usaha)")
    
    else:
        # --- LEVEL 2: FALLBACK DESA ---
        found_in_level_2 = False
        if desa_terdeteksi:
            print(f"-> GAGAL LEVEL 1. Coba Desa {desa_terdeteksi}...", end=" ")
            keyword_2 = f"Kantor Desa {desa_terdeteksi} Kecamatan {KECAMATAN_DEFAULT} Banyuwangi"
            lat, lon = search_google_maps(driver, keyword_2)
            
            if lat:
                print("-> OK (Titik Desa)")
                final_lats.append(f"'{lat}")
                final_lons.append(f"'{lon}")
                keterangan_sumber.append(f"Estimasi (Desa {desa_terdeteksi})")
                found_in_level_2 = True
        
        # --- LEVEL 3: FALLBACK KECAMATAN ---
        if not found_in_level_2:
            print("-> GAGAL LEVEL 2. Coba Kecamatan...", end=" ")
            keyword_3 = f"Kantor Kecamatan {KECAMATAN_DEFAULT} Banyuwangi"
            lat, lon = search_google_maps(driver, keyword_3)
            
            if lat:
                print("-> OK (Titik Kecamatan)")
                final_lats.append(f"'{lat}")
                final_lons.append(f"'{lon}")
                keterangan_sumber.append(f"Estimasi (Kecamatan {KECAMATAN_DEFAULT})")
            else:
                print("-> MANUAL AJA")
                final_lats.append("Cek Manual")
                final_lons.append("Cek Manual")
                keterangan_sumber.append("Gagal Total")

    # Jeda Antar Data
    time.sleep(2.5) 

# --- SAVE ---
df["Latitude_Hasil"] = final_lats
df["Longitude_Hasil"] = final_lons
df["Keterangan_Sumber"] = keterangan_sumber

df.to_excel(output_file, index=False)
driver.quit()
print(f"\nSelesai! File output: {output_file}")

=== START SCRAPING: AUTO DETECT DESA (SAFE MODE) ===
[1] BADAN KERJASAMA ANTAR DESA... -> GAGAL. Coba Kecamatan... -> OK (Titik Kecamatan)
[2] PEDAGANG BAJU... -> OK (Usaha)
[3] MI FATHUL ULUM... -> OK (Usaha)
[4] RA FATHUL ULUM... -> OK (Usaha)
[5] MEUBEL UD JAYA ABADI... -> OK (Usaha)
[6] MEUBEL (HAIDA)... -> OK (Usaha)
[7] MASYARAKAT DESA HUTAN KEMUNING ASRI... -> OK (Usaha)
[8] MASYARAKAT DESA HUTAN RUKUN MAKMUR... -> OK (Usaha)
[9] TK DARUSSALAM KALIPURO... -> OK (Usaha)
[10] MASJID BAITUL MUTTAQIN... -> OK (Usaha)
[11] PERKUMPULAN POKTAN TALANG JERUK... -> OK (Usaha)
[12] PEND.MI DARUSSALAM II... -> OK (Usaha)
[13] KUB DAPUR PIETS... -> OK (Usaha)
[14] PERKUMPULAN KELOMPOK TANI PAPRING KALIPURO... -> OK (Usaha)
[15] MITRA TIRTA LESTARI... -> OK (Usaha)
[16] PUSAT KEGIATAN BELAJAR MASYARAKAT NUR SURYA EDUCATION BANYUWANGI... -> OK (Usaha)
[17] UMAT SEJAHTERA... -> OK (Usaha)
[18] PENDIDIKAN ISLAM DAN SOSIAL SYAMSUL HUDA... -> OK (Usaha)
[19] PESANTREN DARUL MUHLISIN... -> OK (Usah